## Chapter 8: transformación de datos, unir, combinar y reformar
En muchas aplicaciones, los datos pueden extenderse a través de una serie de archivos o bases de datos, o organizarse
en un formulario que no es conveniente analizar.

In [1]:
import pandas as pd
import numpy as np
import sys

## Indexación jerárquica
La indexación jerárquica es una carácteristica importante de Pandas que le permite tener múltiples niveles de índice
en un eje. Otra forma de pensar al respecto es que proporciona una forma de trabajar con datos dimensionales
superiores en una forma dimensional inferior.

In [2]:
data = pd.Series(np.random.uniform(size=9),
                index=[list("aaabbccdd"),
                [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.131928
   2    0.829817
   3    0.144409
b  1    0.719317
   3    0.375293
c  1    0.753454
   2    0.781882
d  2    0.226612
   3    0.024419
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data["b"]

1    0.719317
3    0.375293
dtype: float64

In [5]:
data["b":"c"]

b  1    0.719317
   3    0.375293
c  1    0.753454
   2    0.781882
dtype: float64

In [6]:
data.loc[["b", "d"]]

b  1    0.719317
   3    0.375293
d  2    0.226612
   3    0.024419
dtype: float64

La selección es incluso posible desde un nivel "inner". Seleccionar todos los valores que tiene el valor `2` desde
el segundo nivel de índice:

In [7]:
data.loc[:, 2]

a    0.829817
c    0.781882
d    0.226612
dtype: float64

La indexación jerárquica juega un papel importante en la remodelación de datos y operaciones basadas en grupos,
como formar una tabla pivote. Por ejemplo, puede reorganizar estos datos en un DAtaFrame utilizando el método
`unstack`:

In [8]:
data.unstack()

,1,2,3
a,0.131928,0.829817,0.144409
b,0.719317,NaN,0.375293
c,0.753454,0.781882,NaN
d,NaN,0.226612,0.024419


La operación inversa es `stack`:

In [9]:
inv = data.unstack()
inv.stack()

a  1    0.131928
   2    0.829817
   3    0.144409
b  1    0.719317
   3    0.375293
c  1    0.753454
   2    0.781882
d  2    0.226612
   3    0.024419
dtype: float64

Con un DataFrame, cualquier eje puede tener un índice jerárquico:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[list("aabb"), [1, 2, 1, 2]],
                    columns=[["Ohio", "Ohio", "Colorado"],
                            ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Los niveles jerárquicos pueden tener nombres (como cadenas o cualquier objeto Python). Si es así, esto aparecerán
en la salida de la consola:

In [11]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Esto nombres reemplazan al atributo `name`, que se usa solo con índices de un solo nivel.

Usar `nlevels` para ver cuántos niveles tiene un índice:

In [12]:
frame.index.nlevels

2

Con la indexación parcial de columnas, puede seleccionar de manera similar grupos de columnas:

In [13]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

El DataFrame anterior se puede recrear con `MultiIndex`:

In [14]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green","Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Niveles de reordenación y clasificación
A veces es posible que deba reorganizar el orden de los niveles en un eje u ordenar los datos por los valores en
un nivel específico. El método `swaplevel` toma dos números de nivel o nombres y devuelve un nuevo objeto con los
niveles intercambiados (pero los datos no están alterados):

In [15]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` de forma predeterminada, ordena los datos lexicográficamente utilizando todos los niveles de índice,
pero puede elegir usar solo un nivel único o un subconjunto de niveles para ordenar usando el argumento `level`:

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Resumen estadísticas por nivel
Muchas estadísticas descriptivas y resumidas sobre DataFrame y Series tienen una opción `level` en la que pueden
especificar el nivel que desean agragar en un eje en particular. Se puede agregar por nivel en filas o columnas.

In [18]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexación con columnas de DataFrame
No es inusual querer usar una o más columnas de un DataFrame como índice de fila; alternativamente, es posible
que desee mover el índice de fila a las columnas del DataFrame.

In [20]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                     "c": ["one", "one", "one", "two", "two", "two", "two"],
                     "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


La función `set_index` creará un nuevo DataFrame utilizando una o más de sus columnas como índice:

In [21]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

De froma predeterminada, las colummas se elimina del DataFrame, aunque puede dejarlas con `drop=False`:

In [22]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index` hace lo contrario; los niveles de índice jerárquico se mueven a las columnas:

In [23]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Combiar y fusionar conjuntos de datos
Los contenidos en los objetos de Pandas se pueden combinar de varias maneras:

`pandas.merge`: conecta filas en DataFrame en función de una o más claves. Esto sería familiar para los usuarios de
SQL u otras bases de datos relacionales, ya que implementa la operación de base de datos "_join_".

`pandas.concat`: concatenar o apilar objetos juntos a lo largo de un eje.

`combine_first`: dividir los datos superpuestos para completar los valores faltantes en un objeto con calores de otro.

Las operaciones `merge` o `join` combinan conjuntos de datos al vincular filas usando una o más claves. Estas
operaciones son particularmente importantes en bases de datos relaciones. La función `pandas.merge` es el principal
punto de entrada para usar estos algoritmos en sus datos.

In [24]:
df1 = pd.DataFrame({"key": list("bbacaab"),
                   "data1": pd.Series(range(7), dtype="Int64")})

df2 = pd.DataFrame({"key": list("abd"),
                   "data2": pd.Series(range(3), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [26]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Si no se especifica la columna a unir, `pandas.merge` utiliza los nombres de columna superpuestas como las claves.
Sin embargo, es buena práctica especificar explícitamente:

In [27]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Si lo nombres de la columna son diferentes en cada objeto, puede especificarlos por separado:

In [31]:
df3 = pd.DataFrame({"lkey": list("bbacaab"),
                   "data1": pd.Series(range(7), dtype="Int64")})
df4 = pd.DataFrame({"rkey": list("abd"),
                   "data2": pd.Series(range(3), dtype="Int64")})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Puede notar que a `"c"` y `"d"` le faltan valores y datos asociados en el resultado. Por defecto, `pd.merge` hacer una
unión `"inner"`; las claves en los resultados son las intersecciones, o el conjunto común que se encuentra en ambas
tablas. Otra opciones posibles son `"left"`, `"right"` y `"outer"`. La unión externa toma la unión de las claves,
combinando el efecto de aplicar ambas uniones izquierda y derecha:

In [32]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [33]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


Las fusiones _muchos a muchos_ forman el producto cartesiano de las claves.

In [35]:
df1 = pd.DataFrame({"key": list("bbacab"),
                   "data1": pd.Series(range(6), dtype="Int64")})

df2 = pd.DataFrame({"key": list("abadb"),
                   "data2": pd.Series(range(5), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [36]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,d,3
4,b,4


In [37]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,4
2,b,1,1
3,b,1,4
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [38]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,4
2,b,1,1
3,b,1,4
4,b,5,1
5,b,5,4
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [42]:
# pasar una lista para fusionar varias líneas
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                    "key2": ["one", "two", "one"],
                    "lval": pd.Series(range(3), dtype="Int64")})

right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                     "key2": ["one", "one", "one", "two"],
                     "rval": pd.Series(range(4, 8, 1), dtype="Int64")})

pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,0,4
1,foo,one,0,5
2,foo,two,1,<NA>
3,bar,one,2,6
4,bar,two,<NA>,7


In [44]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,0,one,4
1,foo,one,0,one,5
2,foo,two,1,one,4
3,foo,two,1,one,5
4,bar,one,2,one,6
5,bar,one,2,two,7


`pd.merge` tiene una opción `suffixes` para especificar cadenas para agregar nombres superpuestos en los objetos
DataFrame izquiero y derecho:

In [45]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,0,one,4
1,foo,one,0,one,5
2,foo,two,1,one,4
3,foo,two,1,one,5
4,bar,one,2,one,6
5,bar,one,2,two,7


### Fusión en el índice
En algunos casos, la claves de fusión en un DataFrame se encontrará en su índice. En ese caso, puede pasar
`left_index=True` o `right_index=True` (o ambos) para indicar que el índice debe usarse como la clave de fusión:

In [52]:
left1 = pd.DataFrame({"key": list("abaabc"),
                    "value": pd.Series(range(6), dtype="Int64")})

right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=list("ab"))

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [49]:
right1

,group_val
a,3.5
b,7.0


In [50]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [53]:
# formar la unión de ellas con una unión externa
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Con datos inexados jerárquicamente, las cosas son más complicadas, ya que unir al índice es equivalente a una fusión
de múltiples claves:

In [67]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
                    "key2": [2000, 2001, 2002, 2001, 2002],
                    "data": pd.Series(range(5), dtype="Int64")})

right_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

righth = pd.DataFrame({"event1": pd.Series(range(0, 11, 2), dtype="Int64",
                                          index=right_index),
                     "event2": pd.Series(range(1, 12, 2), dtype="Int64",
                                         index=right_index)})

lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [64]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

En este caso, debe indicar varias columnas para fusionarse como una lista (tener en cuenta el manejo de valores de
índice duplicados con `"outer"`):

In [68]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [69]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
        right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


También es posible usar los índices de ambos lados de la fusión:

In [76]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=list("ace"),
                    columns=["Ohio", "Nevada"]).astype("Int64")

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=list("bcde"),
                     columns=["Missouri", "Alabama"]).astype("Int64")

left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [77]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14
